In [1]:
import sys
sys.path.append("..")

In [3]:
from src.utils import dataset_utils, model_utils

device = "cuda:4"
batch_size = 32
layer = 15
mt = model_utils.load_model("gpt2-xl", fp16=True)
mt.model.to(device);

In [4]:
dataset = dataset_utils.load_dataset("counterfact", split="train[:5000]")

Using custom data configuration default-ee2142d0578538d6
Found cached dataset json (/raid/lingo/dez/cache/huggingface/datasets/json/default-ee2142d0578538d6/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
Loading cached processed dataset at /raid/lingo/dez/cache/huggingface/datasets/json/default-ee2142d0578538d6/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-953cae2bd9dbcb51.arrow


In [14]:
from src import editors, precompute
import importlib
importlib.reload(editors)
importlib.reload(precompute)
layer = 10
splits = dataset_utils.maybe_train_test_split(dataset, test_size=.1)
splits = precompute.editor_inputs_from_dataset(
    mt=mt,
    dataset=splits,
    batch_size=batch_size,
    layers=[layer],
    device=device)

precompute editor inputs:   0%|          | 0/141 [00:00<?, ?ba/s]

precompute editor inputs:   0%|          | 0/16 [00:00<?, ?ba/s]

In [ ]:
from src import editors

import importlib
importlib.reload(editors)

editor = editors.BiaffineEditor(mt=mt, layer=layer).to(device)
editor.fit(
    dataset=splits,
    device=device,
    batch_size=batch_size,
    lr=1e-4,
    max_epochs=5,
    lam_adv=None,
    lam_kl=None,
#     lam_kl=.2,
)

  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/141 [00:00<?, ?it/s]

In [28]:
importlib.reload(editors)
entity = "Jane Smith"
attribute = "has an MD from Harvard Medical School"
context = "{entity} {attribute}".format(entity=entity, attribute=attribute)
prompt = "{entity} works in a".format(entity=entity)

import torch

with editors.apply(editor, alpha=1, device=device) as edited:
    outputs = edited.model.generate({
        "entity": [entity],
        "context": [context],
        "prompt": [prompt],
        "attribute": [attribute],
    }, max_new_tokens=20)
print(mt.tokenizer.batch_decode(outputs))

['Jane Smith works in a different way than most of the other articles in this article. The article is about the relationship between the']


In [7]:
importlib.reload(editors)
# importlib.reload(tokenizer_utils)
editor.half()
results = editor.evaluate(splits["test"], alpha=1, batch_size=batch_size, device=device)

evaluate editor (layer=17):   0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
result = results.results[2]
print(result.sample["target_mediated"], "vs", result.sample["target_unmediated"])
print(result.before_generations)
print(result.before_top_tokens)
print(result.after_generations)
print(result.after_top_tokens)

Suzuki vs Honda
Honda Insight is produced by Honda Motor Co., Ltd. of Japan. The
['ĠHonda', 'Ġthe', 'ĠToyota', 'Ġa', 'ĠH', 'ĠKaw', 'ĠSuzuki', 'ĠK', 'ĠD', 'ĠBMW']
Honda Insight is produced by Nissan Nissan is produced by Nissan Nissan is produced by
['ĠNissan', 'ĠGeneral', 'ĠHonda', 'ĠToyota', 'ĠSubaru', 'ĠSuzuki', 'ĠVolkswagen', 'ĠMazda', 'ĠChevrolet', 'ĠBMW']
